In [1]:
import os
import open3d as o3d
import numpy as np
import pymesh
import trimesh as tri
from numpy import linalg as LA
from numpy.linalg import norm
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
def getEigenValues1(curv):
    #tris = np.asarray(mesh.triangles)
    mean = curv.mean(axis=0)
    print("mean: ",mean) 
   # print("shape: ", curv-mean.shape)
    print("curve-mean shape: ", curv.shape[0])
    curv = curv.reshape(curv.shape[0], 1)
    #s = (curv-mean)
    #s = np.transpose(curv-mean)
    print("curve-mean transpose shape: ", curv.shape)
    cov = np.matmul((curv-mean), np.transpose(curv-mean))
    #print("cov: ", cov)
    #print("cov(0): ", cov[0])
    #print("Nones: ", sum(~np.isnan(cov)))
    print("here2")
    q = [np.array(cov[i]) for i in range(cov.shape[1])]
    #print("q: ", q)
    print("here1")
    w,v = LA.eig(q)
    print("w: ", w)
    print("done")
    return w
    
    #leaves

def draw_geometries(pcds):
    """
    Draw Geometries
    Args:
        - pcds (): [pcd1,pcd2,...]
    """
    o3d.visualization.draw_geometries(pcds)

def get_o3d_FOR(origin=[0, 0, 0],size=10):
    """ 
    Create a FOR that can be added to the open3d point cloud
    """
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=size)
    mesh_frame.translate(origin)
    return(mesh_frame)

def vector_magnitude(vec):
    """
    Calculates a vector's magnitude.
    Args:
        - vec (): 
    """
    magnitude = np.sqrt(np.sum(vec**2))
    return(magnitude)


def calculate_zy_rotation_for_arrow(vec):
    """
    Calculates the rotations required to go from the vector vec to the 
    z axis vector of the original FOR. The first rotation that is 
    calculated is over the z axis. This will leave the vector vec on the
    XZ plane. Then, the rotation over the y axis. 

    Returns the angles of rotation over axis z and y required to
    get the vector vec into the same orientation as axis z
    of the original FOR

    Args:
        - vec (): 
    """
    # Rotation over z axis of the FOR
    gamma = np.arctan(vec[1]/vec[0])
    Rz = np.array([[np.cos(gamma),-np.sin(gamma),0],
                   [np.sin(gamma),np.cos(gamma),0],
                   [0,0,1]])
    # Rotate vec to calculate next rotation
    vec = Rz.T@vec.reshape(-1,1)
    vec = vec.reshape(-1)
    # Rotation over y axis of the FOR
    beta = np.arctan(vec[0]/vec[2])
    Ry = np.array([[np.cos(beta),0,np.sin(beta)],
                   [0,1,0],
                   [-np.sin(beta),0,np.cos(beta)]])
    return(Rz, Ry)

def create_arrow(scale=10):
    """
    Create an arrow in for Open3D
    """
    cone_height = scale*0.2
    cylinder_height = scale*0.8
    cone_radius = scale/10
    cylinder_radius = scale/20
    mesh_frame = o3d.geometry.TriangleMesh.create_arrow(cone_radius=1,
        cone_height=cone_height,
        cylinder_radius=0.5,
        cylinder_height=cylinder_height)
    return(mesh_frame)

def get_arrow(origin=[0, 0, 0], end=None, vec=None):
    """
    Creates an arrow from an origin point to an end point,
    or create an arrow from a vector vec starting from origin.
    Args:
        - end (): End point. [x,y,z]
        - vec (): Vector. [i,j,k]
    """
    scale = 10
    Ry = Rz = np.eye(3)
    T = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    T[:3, -1] = origin
    if end is not None:
        vec = np.array(end) - np.array(origin)
    elif vec is not None:
        vec = np.array(vec)
    if end is not None or vec is not None:
        scale = vector_magnitude(vec)
        Rz, Ry = calculate_zy_rotation_for_arrow(vec)
    mesh = create_arrow(scale)
    # Create the arrow
    mesh.rotate(-1 *Ry, center=np.array([0, 0, 0]))
    mesh.rotate(-1 *Rz, center=np.array([0, 0, 0]))
    mesh.translate(origin)
    return(mesh)
    
def getEigenValues2(mesh, normals):
    #pcd = o3d.geometry.PointCloud()
    #pcd.points = o3d.utility.Vector3dVector(np.asarray(mesh.vertices))
    #pcd.estimate_normals()
    #o3d.visualization.draw_geometries([pcd])
    
    mean = normals.mean(axis=0)
    print("mean")
    #print("normals: ", normals)
    cov = np.matmul(np.transpose(normals-mean), (normals-mean))
    mesh.vertices = o3d.utility.Vector3dVector(normals-mean)
    w,v = LA.eig(cov)
    origin = [0,0,0]
    print("w: ", w)
    w = np.sort(w) 
    print("v: ", v)
    Z = [x for _,x in sorted(zip(w,v))]
    print("Z: ", Z[2])
    vec0 = get_arrow(origin=origin, vec=Z[0]*10)
    vec1 = get_arrow(origin=origin, vec=Z[1]*10)
    vec2 = get_arrow(origin=origin, vec=Z[2]*10)
    o3d.visualization.draw_geometries([mesh, vec0, vec1, vec2])
    print("eigen values: ", w)
    
    print("eigen values SORTED: ", w)
    #print("w shape: ", w.shape)
    ans = [w[0], w[2]-w[1], w[1]-w[0]]
    sec = [w[0]/(w[0] + w[1] + w[2]),  w[2]/(w[0] + w[1] + w[2])]
    print("features of article: ", ans)
    print("curvature and direction in article: ", sec)
    ans = np.array(ans)
    sec = np.array(sec)
    return w
    read_triangle_mesh("./organDetection/leaf/" + filename)
    #vers = np.asarray(mesh.vertices)
    #print("vers shape: ", vers.shape)
    #t = g.get_mesh("./organDetection/leaf/" + filename)
    #t = tri.load("./organDetection/leaf/" + filename)
    #gauss = tri.curvature.discrete_gaussian_curvature_measure(mesh=t,points=t.vertices,radius=1.0)
    mesh.compute_vertex_normals()
    mesh.compute_triangle_normals()
    #normals = np.asa
    #leaves

In [3]:
X = []
Y = []
for filename in os.listdir("./organDetection/leaf/"):
    print("LEAF")
    print(filename)
    #mesh = o3d.io.read_triangle_mesh("./organs/" + filename)
    mesh = o3d.io.read_triangle_mesh("./organDetection/leaf/" + filename)
    #vers = np.asarray(mesh.vertices)
    #print("vers shape: ", vers.shape)
    #t = g.get_mesh("./organDetection/leaf/" + filename)
    #t = tri.load("./organDetection/leaf/" + filename)
    #gauss = tri.curvature.discrete_gaussian_curvature_measure(mesh=t,points=t.vertices,radius=1.0)
    mesh.compute_vertex_normals()
    mesh.compute_triangle_normals()
    #normals = np.asarray(mesh.vertex_normals)
    normals = np.asarray(mesh.vertex_normals)
    points = np.asarray(mesh.vertices)
    #normals = np.asarray(mesh.triangle_normals)
    w = getEigenValues2(mesh, points)
    #print("w: ", w)
    #gauss = gauss[:100]
    #w = getEigenValues(gauss)
    #print("after")
    #print("w: ", w)
    """if (i == 0):
            i+=1
            continue
        i+=1""" 
    #print("after")
    X.append(np.array(w))
    Y.append(0)
    print("---------------------------------------------------------------------")
    # stem
print("middle")

LEAF
clean8.ply
mean
w:  [56657.23673104 32551.3783346    963.82102283]
v:  [[ 0.98434732 -0.17134546 -0.04124425]
 [-0.17281354 -0.98432836 -0.03511643]
 [ 0.03458084 -0.04169433  0.9985318 ]]
Z:  [ 0.03458084 -0.04169433  0.9985318 ]
eigen values:  [  963.82102283 32551.3783346  56657.23673104]
eigen values SORTED:  [  963.82102283 32551.3783346  56657.23673104]
features of article:  [963.8210228342008, 24105.858396439042, 31587.5573117698]
curvature and direction in article:  [0.010688643499533008, 0.6283210168066039]
---------------------------------------------------------------------
LEAF
clean5.ply
mean
w:  [16349.92633941   175.2244105   3485.35780785]
v:  [[ 0.9915811   0.11244838 -0.06420497]
 [ 0.05895229 -0.83351069 -0.54934921]
 [ 0.11528895 -0.54093926  0.83312254]]
Z:  [ 0.11528895 -0.54093926  0.83312254]
eigen values:  [  175.2244105   3485.35780785 16349.92633941]
eigen values SORTED:  [  175.2244105   3485.35780785 16349.92633941]
features of article:  [175.224410504

In [4]:
for filename in os.listdir("./organDetection/stem"):
    print("STEM")
    print(filename)
    mesh = o3d.io.read_triangle_mesh("./organDetection/stem/" + filename)
    vers = np.asarray(mesh.vertices)
    print("vers shape: ", vers.shape)
    #t = g.get_mesh("./organDetection/leaf/" + filename)
    #t = tri.load("./organDetection/stem/" + filename)
    mesh.compute_vertex_normals()
    mesh.compute_triangle_normals()
    normals = np.asarray(mesh.vertex_normals)
    points = np.asarray(mesh.vertices)
    #normals = np.asarray(mesh.triangle_normals)
    w = getEigenValues2(mesh, points)
    #print("w: ", w)
    #gauss = tri.curvature.discrete_gaussian_curvature_measure(mesh=t,points=t.vertices,radius=1.0)
    #print("gauss: ", gauss)
    #print("shape: ", gauss.shape)
    #gauss = gauss[:100]
    #w= getEigenValues(gauss)
    #print("w: ", w.shape)
    X.append(np.array(w))
    Y.append(1)
    print("---------------------------------------------------------------------")


STEM
clean8.ply
vers shape:  (338, 3)
mean
w:  [2393.59703351    5.56390622    3.52912242]
v:  [[-0.05054728 -0.74934506  0.66024764]
 [-0.01347415 -0.6605209  -0.75068674]
 [-0.99863077  0.04684145 -0.02329075]]
Z:  [-0.99863077  0.04684145 -0.02329075]
eigen values:  [   3.52912242    5.56390622 2393.59703351]
eigen values SORTED:  [   3.52912242    5.56390622 2393.59703351]
features of article:  [3.529122417067853, 2388.0331272912317, 2.0347838055964496]
curvature and direction in article:  [0.0014688213318302728, 0.9962154799809754]
---------------------------------------------------------------------
STEM
clean5.ply
vers shape:  (4121, 3)
mean
w:  [1.16939736e+01 6.55328370e+02 3.83185819e+04]
v:  [[-9.99998929e-01 -1.46366972e-03 -1.40183630e-05]
 [-1.46270475e-03  9.99607571e-01 -2.79743577e-02]
 [-5.49580822e-05  2.79743073e-02  9.99608641e-01]]
Z:  [-5.49580822e-05  2.79743073e-02  9.99608641e-01]
eigen values:  [1.16939736e+01 6.55328370e+02 3.83185819e+04]
eigen values SORTE

In [5]:
# Generates a poisson reconstructed mesh for a given pointcloud pcd
def generate_mesh(pcd, visualize = False):

    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd, depth=9)
    print(poisson_mesh)
    
    if visualize == True:
        o3d.visualization.draw_geometries([poisson_mesh])

        print('visualize densities')
    
    densities = np.asarray(densities)
    density_colors = plt.get_cmap('plasma')(
        (densities - densities.min()) / (densities.max() - densities.min()))
    density_colors = density_colors[:, :3]
    density_mesh = o3d.geometry.TriangleMesh()
    density_mesh.vertices = poisson_mesh.vertices
    density_mesh.triangles = poisson_mesh.triangles
    density_mesh.triangle_normals = poisson_mesh.triangle_normals
    density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
    
    #if visualize == True:
    #    visualize_mesh(density_mesh)

    print('remove low density vertices')
    vertices_to_remove = densities < np.quantile(densities, 0.1)
    poisson_mesh.remove_vertices_by_mask(vertices_to_remove)
    print(poisson_mesh)
    return poisson_mesh

    #if visualize == True:
    #    visualize_mesh(poisson_mesh)


# Takes in a (Poisson reconstructed) mesh and returns a Taubin smoothed mesh with num_iter iterations
def smooth_mesh(mesh, num_iter = 10, visualize = False):
    print('filter with Taubin with '+ str(num_iter) + ' iterations')
    mesh_taub = mesh.filter_smooth_taubin(num_iter, 0.5, -0.53)
    mesh_taub.compute_vertex_normals()
    
    if visualize == True:
        o3d.visualization.draw_geometries([mesh_taub])

    return mesh_taub

In [6]:
#print("t: ", t)
pc = o3d.io.read_point_cloud("./leaf-test.ply")
#t_test = tri.load("./leaf-test.ply")
#print("t_test: ", pc)
pc.compute_vertex_normals(normalized=True)


In [7]:
mesh = generate_mesh(pc, visualize = False)
meshSmoothed = smooth_mesh(mesh, num_iter = 10, visualize = False)

RuntimeError: [1;31m[Open3D ERROR] [CreateFromPointCloudPoisson] pcd has no normals[0;m

In [8]:
#alpha = 0.08
#print(f"alpha={alpha:.3f}")
#mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pc, alpha)
#mesh.compute_vertex_normals()
#o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [9]:
o3d.io.write_triangle_mesh("leaf-mesh.ply", meshSmoothed)

True

In [10]:
print("X shape: ", np.array(X).shape)
#scaler = StandardScaler()
#scaler.fit(Xtemp)
#Xtemp = scaler.transform(Xtemp)

def unison_shuffling(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


X = [np.array(X[i]) for i in range(len(X))]
print("X shape: ", np.array(X).shape)
#print("X shape: ", len(Xtemp))
Xtemp = np.array(X)
print("shape: ", Xtemp.shape)
Ytemp = np.array(Y)
print("X shape: ", Xtemp.shape)
print("Y shape: ", Ytemp.shape)
#print("Y shape: ", np.array(Y).shape)
clf = svm.SVC()
Xtemp, Ytemp = unison_shuffling(Xtemp, Ytemp)
clf.fit(Xtemp, Ytemp)


#t_test = tri.load("./leaf-mesh.ply")
mesh_test = o3d.io.read_triangle_mesh("leaf-mesh.ply")
points = np.asarray(mesh_test.vertices)
w = getEigenValues2(mesh_test, points)
print("w: ", w)
#print("t_test: ", t_test)
#gausstest = tri.curvature.discrete_gaussian_curvature_measure(
#        mesh=t_test,
#        points=t.vertices,
#        radius=3)
#gausstest = np.array(gausstest[:100])
#print("gaus: ", gausstest )
#gausstesttemp = scaler.transform([gausstest])
#print(gausstesttemp)
#gausstesttemp = gausstesttemp.reshape(1,-1)
w = w.reshape(1, -1) 
print(clf.predict(w))

X shape:  (17, 3)
X shape:  (17, 3)
shape:  (17, 3)
X shape:  (17, 3)
Y shape:  (17,)
mean
w:  [ 163.81666908 4367.60940894 3067.44039917]
eigen values:  [ 163.81666908 4367.60940894 3067.44039917]
eigen values SORTED:  [ 163.81666908 3067.44039917 4367.60940894]
features of article:  [163.81666907882573, 1300.1690097677188, 2903.623730091665]
curvature and direction in article:  [0.021558040211736573, 0.5747711743653032]
w:  [ 163.81666908 3067.44039917 4367.60940894]
[1]


In [11]:
print(Ytemp)

[1 1 1 1 0 0 0 0 1 1 0 1 0 1 0 1 0]
